## 1. セットアップ

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from pathlib import Path
import cv2
from collections import Counter

sys.path.append('/workspace/atma_22_ca/')
from configs.config import *

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"Input dir: {DIR_INPUT}")
print(f"Train crops dir: {DIR_TRAIN_CROPS}")
print(f"Test crops dir: {DIR_CROPS}")

## 2. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

# df_test.rename(columns={'session_no': 'session'}, inplace=True)
df_test.rename(columns={'frame_in_session': 'frame'}, inplace=True)

print(f"訓練データ: {df_train.shape}")
print(f"テストデータ: {df_test.shape}")
print(f"\n訓練データ列: {df_train.columns.tolist()}")
print(f"テストデータ列: {df_test.columns.tolist()}")

In [ ]:
# 最初の数行を確認
print("=== 訓練データ ===")
display(df_train.head())

print("\n=== テストデータ ===")
display(df_test.head())

## 3. 基本統計

In [ ]:
# 訓練データの基本統計
print("=== 訓練データ基本統計 ===")
print(f"総サンプル数: {len(df_train):,}")
print(f"選手数: {df_train['label_id'].nunique()}")
print(f"Quarter数: {df_train['quarter'].nunique()}")
print(f"Session数: {df_train['session'].nunique()}")
print(f"画角: {df_train['angle'].unique()}")
print(f"\n選手ごとのサンプル数:")
print(df_train['label_id'].value_counts().sort_index())

In [ ]:
# テストデータの基本統計
print("=== テストデータ基本統計 ===")
print(f"総サンプル数: {len(df_test):,}")
print(f"Quarter数: {df_test['quarter'].nunique()}")
print(f"Session数: {df_test['session_no'].nunique()}")
print(f"画角: {df_test['angle'].unique()}")
print(f"\nrel_path列の最初の5件:")
print(df_test['rel_path'].head())

## 4. 選手分布の可視化

In [ ]:
# 選手ごとのサンプル数
plt.figure(figsize=(12, 5))
label_counts = df_train['label_id'].value_counts().sort_index()
plt.bar(label_counts.index, label_counts.values, color='skyblue', edgecolor='black')
plt.xlabel('Player ID', fontsize=12)
plt.ylabel('Sample Count', fontsize=12)
plt.title('Training Data: Sample Distribution per Player', fontsize=14)
plt.xticks(label_counts.index)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"最小サンプル数: {label_counts.min()}")
print(f"最大サンプル数: {label_counts.max()}")
print(f"平均サンプル数: {label_counts.mean():.1f}")

## 5. Quarter・Session分布

In [ ]:
# 訓練データのQuarter分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quarter分布
quarter_counts_train = df_train['quarter'].value_counts().sort_index()
axes[0].bar(range(len(quarter_counts_train)), quarter_counts_train.values, color='lightcoral', edgecolor='black')
axes[0].set_xlabel('Quarter', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Training Data: Quarter Distribution', fontsize=14)
axes[0].set_xticks(range(len(quarter_counts_train)))
axes[0].set_xticklabels(quarter_counts_train.index, rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Session分布（訓練データは全てsession=0）
session_counts_train = df_train['session'].value_counts().sort_index()
axes[1].bar(session_counts_train.index, session_counts_train.values, color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Session', fontsize=12)
axes[1].set_ylabel('Sample Count', fontsize=12)
axes[1].set_title('Training Data: Session Distribution', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# テストデータのQuarter・Session分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quarter分布
quarter_counts_test = df_test['quarter'].value_counts().sort_index()
axes[0].bar(range(len(quarter_counts_test)), quarter_counts_test.values, color='lightcoral', edgecolor='black')
axes[0].set_xlabel('Quarter', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Test Data: Quarter Distribution', fontsize=14)
axes[0].set_xticks(range(len(quarter_counts_test)))
axes[0].set_xticklabels(quarter_counts_test.index, rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Session分布（テストデータは複数sessionがある）
session_counts_test = df_test['session_no'].value_counts().sort_index()
axes[1].bar(session_counts_test.index, session_counts_test.values, color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Session', fontsize=12)
axes[1].set_ylabel('Sample Count', fontsize=12)
axes[1].set_title('Test Data: Session Distribution', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"テストデータのSession数: {df_test['session_no'].nunique()}")
print(f"各Sessionのサンプル数:\n{session_counts_test}")

In [ ]:
# uarter・Sessionごとの出場選手の分析

# 訓練データ: Quarterごとの選手分布
print("=== 訓練データ: Quarterごとの選手出場状況 ===")
quarter_label_matrix = df_train.groupby(['quarter', 'label_id']).size().unstack(fill_value=0)

# ヒートマップで可視化
plt.figure(figsize=(14, 10))
sns.heatmap(quarter_label_matrix.T, cmap='YlOrRd', annot=True, fmt='d', cbar_kws={'label': 'Sample Count'})
plt.xlabel('Quarter', fontsize=12)
plt.ylabel('Player ID', fontsize=12)
plt.title('Training Data: Player Appearance per Quarter', fontsize=14)
plt.tight_layout()
plt.show()

# テストデータ: Quarter・Sessionの組み合わせ確認
print("\n=== テストデータ: Quarter × Session の組み合わせ ===")
test_quarter_session = df_test.groupby(['quarter', 'session_no']).size().reset_index(name='count')
print(test_quarter_session.head(20))
print(f"\n総組み合わせ数: {len(test_quarter_session)}")

In [ ]:
# 訓練データ: Quarter × Frame の分布
print("=== 訓練データ: Quarter × Frame 分布 ===")
train_quarter_frame = df_train.groupby(['quarter', 'frame']).size().reset_index(name='count')
print(f"総フレーム数: {df_train['frame'].nunique()}")
print(f"Quarter別フレーム数:")
for quarter in sorted(df_train['quarter'].unique()):
    quarter_data = df_train[df_train['quarter'] == quarter]
    print(f"  Quarter {quarter}: {quarter_data['frame'].nunique()} フレーム, "
          f"Frame範囲: {quarter_data['frame'].min()}-{quarter_data['frame'].max()}")

# 可視化: Quarterごとのフレーム数分布
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
quarters = sorted(df_train['quarter'].unique())

for i, quarter in enumerate(quarters[:4]):
    ax = axes[i // 2, i % 2]
    quarter_data = df_train[df_train['quarter'] == quarter]
    frame_counts = quarter_data['frame'].value_counts().sort_index()
    
    ax.plot(frame_counts.index, frame_counts.values, marker='o', markersize=2, alpha=0.7)
    ax.set_xlabel('Frame Number', fontsize=11)
    ax.set_ylabel('BBox Count per Frame', fontsize=11)
    ax.set_title(f'Quarter {quarter}: Frame分布 ({len(frame_counts)} frames)', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n各Quarterの統計:")
print(train_quarter_frame.groupby('quarter')['count'].describe())

In [ ]:
# テストデータ: Quarter × Session × Frame の分布
print("=== テストデータ: Quarter × Session × Frame 分布 ===")
test_quarter_session_frame = df_test.groupby(['quarter', 'session_no', 'frame']).size().reset_index(name='count')
print(f"総フレーム数: {df_test['frame'].nunique()}")
print(f"\nQuarter × Session別フレーム数（上位20件）:")
for _, row in test_quarter_session_frame.groupby(['quarter', 'session_no'])['frame'].nunique().reset_index(name='frame_count').head(20).iterrows():
    print(f"  Quarter {row['quarter']}, Session {row['session_no']}: {row['frame_count']} フレーム")

# テストデータ: 各Quarter × Sessionごとのフレーム範囲
print("\n=== 各Quarter × Sessionのフレーム範囲（サンプル） ===")
session_frame_info = df_test.groupby(['quarter', 'session_no'])['frame'].agg(['min', 'max', 'count', 'nunique']).reset_index()
session_frame_info.columns = ['quarter', 'session_no', 'frame_min', 'frame_max', 'bbox_count', 'frame_count']
session_frame_info['frame_range'] = session_frame_info['frame_max'] - session_frame_info['frame_min'] + 1
print(session_frame_info.head(15))

In [ ]:
# テストデータ: セッションごとのフレーム分布を可視化
print("=== テストデータ: Session別フレーム分布サンプル ===")
# サンプルとして4つのセッションを可視化
sample_sessions = df_test.groupby(['quarter', 'session_no']).size().reset_index(name='count').sort_values('count', ascending=False).head(4)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, (_, session_info) in enumerate(sample_sessions.iterrows()):
    quarter = session_info['quarter']
    session = session_info['session_no']
    
    # 該当セッションのデータ抽出
    session_data = df_test[(df_test['quarter'] == quarter) & (df_test['session_no'] == session)]
    frame_counts = session_data['frame'].value_counts().sort_index()
    
    axes[i].plot(frame_counts.index, frame_counts.values, marker='o', markersize=3, alpha=0.7, color='orangered')
    axes[i].set_xlabel('Frame Number', fontsize=11)
    axes[i].set_ylabel('BBox Count per Frame', fontsize=11)
    axes[i].set_title(f'Quarter {quarter}, Session {session}\n{len(frame_counts)} frames, {session_info["count"]} bboxes', 
                     fontsize=12, fontweight='bold')
    axes[i].grid(alpha=0.3)
    
    # フレーム範囲を表示
    axes[i].text(0.02, 0.98, f'Frame: {frame_counts.index.min()}-{frame_counts.index.max()}', 
                transform=axes[i].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

In [ ]:
# フレームごとのBBox数の分布（訓練 vs テスト）
print("=== フレームごとのBBox数の分布 ===")

# 訓練データ: フレームごとのBBox数
train_frame_bbox_counts = df_train.groupby('frame').size()
print(f"訓練データ:")
print(f"  平均BBox数/Frame: {train_frame_bbox_counts.mean():.2f}")
print(f"  最小BBox数/Frame: {train_frame_bbox_counts.min()}")
print(f"  最大BBox数/Frame: {train_frame_bbox_counts.max()}")
print(f"  中央値BBox数/Frame: {train_frame_bbox_counts.median():.2f}")

# テストデータ: フレームごとのBBox数
test_frame_bbox_counts = df_test.groupby(['quarter', 'session_no', 'frame']).size()
print(f"\nテストデータ:")
print(f"  平均BBox数/Frame: {test_frame_bbox_counts.mean():.2f}")
print(f"  最小BBox数/Frame: {test_frame_bbox_counts.min()}")
print(f"  最大BBox数/Frame: {test_frame_bbox_counts.max()}")
print(f"  中央値BBox数/Frame: {test_frame_bbox_counts.median():.2f}")

# ヒストグラムで比較
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(train_frame_bbox_counts, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(train_frame_bbox_counts.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {train_frame_bbox_counts.mean():.1f}')
axes[0].axvline(train_frame_bbox_counts.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {train_frame_bbox_counts.median():.1f}')
axes[0].set_xlabel('BBox Count per Frame', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Training Data: BBox Count Distribution', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

axes[1].hist(test_frame_bbox_counts, bins=30, color='orange', edgecolor='black', alpha=0.7)
axes[1].axvline(test_frame_bbox_counts.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {test_frame_bbox_counts.mean():.1f}')
axes[1].axvline(test_frame_bbox_counts.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {test_frame_bbox_counts.median():.1f}')
axes[1].set_xlabel('BBox Count per Frame', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Test Data: BBox Count Distribution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# フレーム間での選手の連続性（訓練データ）
print("=== フレーム間での選手の連続性分析 ===")

# 各Quarterで、連続するフレーム間での選手変化を確認
for quarter in sorted(df_train['quarter'].unique())[:2]:  # 最初の2つのQuarterのみ
    quarter_data = df_train[df_train['quarter'] == quarter].sort_values('frame')
    
    # フレームごとの選手セット
    frame_players = quarter_data.groupby('frame')['label_id'].apply(set).to_dict()
    frames = sorted(frame_players.keys())
    
    # 連続するフレーム間での変化を計算
    continuity_scores = []
    for i in range(len(frames) - 1):
        current_frame = frames[i]
        next_frame = frames[i + 1]
        
        current_players = frame_players[current_frame]
        next_players = frame_players[next_frame]
        
        # Jaccard類似度（重なり度合い）
        intersection = len(current_players & next_players)
        union = len(current_players | next_players)
        jaccard = intersection / union if union > 0 else 0
        continuity_scores.append(jaccard)
    
    print(f"\nQuarter {quarter}:")
    print(f"  総フレーム数: {len(frames)}")
    print(f"  平均連続性（Jaccard類似度）: {np.mean(continuity_scores):.3f}")
    print(f"  選手完全一致率: {sum(s == 1.0 for s in continuity_scores) / len(continuity_scores) * 100:.1f}%")
    print(f"  選手変化あり率: {sum(s < 1.0 for s in continuity_scores) / len(continuity_scores) * 100:.1f}%")
    
    # プロット
    plt.figure(figsize=(14, 4))
    plt.plot(range(len(continuity_scores)), continuity_scores, marker='o', markersize=2, alpha=0.6)
    plt.axhline(y=1.0, color='g', linestyle='--', alpha=0.5, label='完全一致')
    plt.axhline(y=np.mean(continuity_scores), color='r', linestyle='--', alpha=0.5, label=f'平均: {np.mean(continuity_scores):.3f}')
    plt.xlabel('Frame Transition Index', fontsize=11)
    plt.ylabel('Jaccard Similarity', fontsize=11)
    plt.title(f'Quarter {quarter}: Frame-to-Frame Player Continuity', fontsize=13, fontweight='bold')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

## 5.2. Quarter + Session + Frame の組み合わせ分析（後処理の制約単位）

In [ ]:
df_train

In [ ]:
df_train[df_train['frame_id']=='Q1-000_0_1']

In [ ]:
# 訓練データ: quarter + session + frame のユニークな組み合わせ
print("=== 訓練データ: Quarter + Session + Frame の組み合わせ ===")
df_train['frame_id'] = df_train['quarter'] + '_' + df_train['session'].astype(str) + '_' + df_train['frame'].astype(str)
train_frame_groups = df_train.groupby('frame_id').agg({
    'label_id': ['count', 'nunique', lambda x: list(x)],
    'quarter': 'first',
    'session': 'first',
    'frame': 'first'
}).reset_index()
train_frame_groups.columns = ['frame_id', 'bbox_count', 'unique_players', 'player_list', 'quarter', 'session', 'frame']

print(f"ユニークなフレーム数: {len(train_frame_groups)}")
print(f"フレームあたりの平均BBox数: {train_frame_groups['bbox_count'].mean():.2f}")
print(f"フレームあたりの平均選手数: {train_frame_groups['unique_players'].mean():.2f}")
print(f"\nBBox数の分布:")
print(train_frame_groups['bbox_count'].describe())
print(f"\nサンプル（最初の10フレーム）:")
print(train_frame_groups[['frame_id', 'bbox_count', 'unique_players', 'quarter', 'session', 'frame']].head(10))

In [ ]:
# テストデータ: quarter + session_no + frame のユニークな組み合わせ
print("=== テストデータ: Quarter + Session + Frame の組み合わせ ===")
df_test['frame_id'] = df_test['quarter'] + '_' + df_test['session_no'].astype(str) + '_' + df_test['frame'].astype(str)
test_frame_groups = df_test.groupby('frame_id').agg({
    'quarter': 'first',
    'session_no': 'first',
    'frame': 'first'
}).reset_index()
test_frame_groups['bbox_count'] = df_test.groupby('frame_id').size().values

print(f"ユニークなフレーム数: {len(test_frame_groups)}")
print(f"フレームあたりの平均BBox数: {test_frame_groups['bbox_count'].mean():.2f}")
print(f"\nBBox数の分布:")
print(test_frame_groups['bbox_count'].describe())
print(f"\nサンプル（最初の10フレーム）:")
print(test_frame_groups[['frame_id', 'bbox_count', 'quarter', 'session_no', 'frame']].head(10))

# 訓練とテストの比較
print(f"\n=== 訓練 vs テスト比較 ===")
print(f"訓練データ: {len(train_frame_groups)} ユニークフレーム")
print(f"テストデータ: {len(test_frame_groups)} ユニークフレーム")
print(f"訓練データ平均BBox数/フレーム: {train_frame_groups['bbox_count'].mean():.2f}")
print(f"テストデータ平均BBox数/フレーム: {test_frame_groups['bbox_count'].mean():.2f}")

In [ ]:
# 訓練データ: 同一フレーム内での選手の重複チェック
print("=== 訓練データ: 同一フレーム内での選手重複パターン ===")

# 各フレームで選手IDの重複があるかチェック
duplicate_analysis = []
for frame_id, group in df_train.groupby('frame_id'):
    player_ids = group['label_id'].tolist()
    unique_players = len(set(player_ids))
    total_bbox = len(player_ids)
    has_duplicate = unique_players < total_bbox
    
    if has_duplicate:
        duplicate_analysis.append({
            'frame_id': frame_id,
            'total_bbox': total_bbox,
            'unique_players': unique_players,
            'duplicates': total_bbox - unique_players,
            'player_list': player_ids
        })

if duplicate_analysis:
    df_duplicates = pd.DataFrame(duplicate_analysis)
    print(f"\n⚠️ 重複あり: {len(df_duplicates)} フレーム")
    print(df_duplicates.head(10))
else:
    print("\n✅ 訓練データ: 全フレームで選手の重複なし（各選手は各フレームで最大1回出現）")

# 後処理制約への示唆
print("\n=== 後処理への示唆 ===")
print("✓ 制約1: 同一フレーム（quarter+session+frame）内で各選手は最大1回出現")
print(f"✓ 訓練データの整合性: {'重複なし' if not duplicate_analysis else '一部重複あり'}")

In [ ]:
# frame_idごとのbbox_count分布を可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 訓練データ: BBox数の分布
train_frame_groups = train_frame_groups.sort_values(by='frame_id')
axes[0].bar(train_frame_groups.frame_id, train_frame_groups.bbox_count, color='skyblue', edgecolor='black', alpha=0.7)
# axes[0].axvline(train_frame_groups['bbox_count'].mean(), color='red', linestyle='--', linewidth=2, 
# 				label=f'平均: {train_frame_groups["bbox_count"].mean():.2f}')
# axes[0].axvline(train_frame_groups['bbox_count'].median(), color='green', linestyle='--', linewidth=2, 
# 				label=f'中央値: {train_frame_groups["bbox_count"].median():.0f}')
axes[0].set_xlabel('フレームあたりのBBox数', fontsize=12)
axes[0].set_ylabel('フレーム数', fontsize=12)
axes[0].set_title('訓練データ: フレームあたりのBBox数分布', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# テストデータ: BBox数の分布
test_frame_groups = test_frame_groups.sort_values(by='frame_id')
axes[1].bar(test_frame_groups.frame_id, test_frame_groups.bbox_count, color='skyblue', edgecolor='black', alpha=0.7)
axes[1].axvline(test_frame_groups['bbox_count'].mean(), color='red', linestyle='--', linewidth=2, 
				label=f'平均: {test_frame_groups["bbox_count"].mean():.2f}')
axes[1].axvline(test_frame_groups['bbox_count'].median(), color='green', linestyle='--', linewidth=2, 
				label=f'中央値: {test_frame_groups["bbox_count"].median():.0f}')
axes[1].set_xlabel('フレームあたりのBBox数', fontsize=12)
axes[1].set_ylabel('フレーム数', fontsize=12)
axes[1].set_title('テストデータ: フレームあたりのBBox数分布', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# フレームごとのBBox数の分布を詳細に可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 訓練データ: BBox数の分布
axes[0, 0].hist(train_frame_groups['bbox_count'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(train_frame_groups['bbox_count'].mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {train_frame_groups["bbox_count"].mean():.2f}')
axes[0, 0].set_xlabel('BBox Count per Frame', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Train: BBox Count Distribution\n(per Quarter+Session+Frame)', fontsize=13, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# テストデータ: BBox数の分布
axes[0, 1].hist(test_frame_groups['bbox_count'], bins=30, color='orange', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(test_frame_groups['bbox_count'].mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {test_frame_groups["bbox_count"].mean():.2f}')
axes[0, 1].set_xlabel('BBox Count per Frame', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Test: BBox Count Distribution\n(per Quarter+Session+Frame)', fontsize=13, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# 訓練データ: ユニーク選手数の分布
axes[1, 0].hist(train_frame_groups['unique_players'], bins=range(1, 15), color='lightgreen', edgecolor='black', alpha=0.7)
axes[1, 0].axvline(train_frame_groups['unique_players'].mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {train_frame_groups["unique_players"].mean():.2f}')
axes[1, 0].set_xlabel('Unique Player Count per Frame', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Train: Unique Player Count Distribution', fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# テストデータ: 予測ターゲット数（BBox数と同じ）
axes[1, 1].hist(test_frame_groups['bbox_count'], bins=range(1, 20), color='salmon', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(test_frame_groups['bbox_count'].mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {test_frame_groups["bbox_count"].mean():.2f}')
axes[1, 1].set_xlabel('Prediction Target Count per Frame', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Test: Prediction Target Distribution\n(各フレームで予測すべきBBox数)', fontsize=13, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n=== 統計サマリー ===")
print(f"訓練: BBox数/フレーム = {train_frame_groups['bbox_count'].mean():.2f} ± {train_frame_groups['bbox_count'].std():.2f}")
print(f"訓練: 選手数/フレーム = {train_frame_groups['unique_players'].mean():.2f} ± {train_frame_groups['unique_players'].std():.2f}")
print(f"テスト: BBox数/フレーム = {test_frame_groups['bbox_count'].mean():.2f} ± {test_frame_groups['bbox_count'].std():.2f}")

In [ ]:
# Quarter × Session ごとの統計
print("=== Quarter × Session ごとの統計（訓練データ） ===")
train_quarter_session_stats = df_train.groupby(['quarter', 'session']).agg({
    'frame': 'nunique',
    'label_id': ['count', 'nunique']
}).reset_index()
train_quarter_session_stats.columns = ['quarter', 'session', 'unique_frames', 'total_bbox', 'unique_players']
train_quarter_session_stats['avg_bbox_per_frame'] = train_quarter_session_stats['total_bbox'] / train_quarter_session_stats['unique_frames']

print(train_quarter_session_stats)
print(f"\n全Quarter合計: {train_quarter_session_stats['unique_frames'].sum()} フレーム")
print(f"全Quarter合計: {train_quarter_session_stats['total_bbox'].sum()} BBox")

# テストデータ: Quarter × Session統計
print("\n=== Quarter × Session ごとの統計（テストデータ） ===")
test_quarter_session_stats = df_test.groupby(['quarter', 'session_no']).agg({
    'frame': 'nunique'
}).reset_index()
test_quarter_session_stats.columns = ['quarter', 'session_no', 'unique_frames']
test_quarter_session_stats['total_bbox'] = df_test.groupby(['quarter', 'session_no']).size().values
test_quarter_session_stats['avg_bbox_per_frame'] = test_quarter_session_stats['total_bbox'] / test_quarter_session_stats['unique_frames']

print(test_quarter_session_stats.head(20))
print(f"\n全Quarter × Session合計: {test_quarter_session_stats['unique_frames'].sum()} フレーム")
print(f"全Quarter × Session合計: {test_quarter_session_stats['total_bbox'].sum()} BBox")

### Quarter + Session + Frame 分析のまとめ

**重要な発見:**

1. **フレームの一意性**:
   - 訓練: `quarter + session + frame` で一意のフレームを識別
   - テスト: `quarter + session_no + frame` で一意のフレームを識別
   - この単位が後処理の最適化問題の **制約の基本単位**

2. **同一フレーム内の制約**:
   - 訓練データでは各フレーム内で選手の重複なし
   - **制約**: 同一フレーム内で各選手IDは最大1回出現

3. **フレームあたりのBBox数**:
   - 訓練: 平均〜10個/フレーム
   - テスト: 平均〜8個/フレーム（検出精度のシフト）
   - **示唆**: テストでは不正確なBBoxにより、実際の選手数より多い/少ない可能性

4. **後処理の最適化問題への適用**:
   - **グルーピング**: `quarter + session + frame` でグループ化
   - **制約適用範囲**: 各グループ内で選手重複禁止
   - **予測ターゲット**: 各グループごとにBBox数分の選手ID予測

5. **Quarter × Session の特性**:
   - 訓練: 各Quarterで session=0 のみ（全フレーム連続）
   - テスト: 各Quarterで複数sessionに分割（断続的なフレーム）
   - **CV戦略**: `quarter + session` でグループ化してGroup K-Fold

### フレーム情報分析のまとめ

**訓練データの特徴:**
- 各Quarterで連続したフレーム番号
- フレームごとに平均10名程度の選手BBox
- 連続フレーム間で選手の入れ替わりが発生（交代を含む）

**テストデータの特徴:**
- Quarter × Sessionごとにフレーム番号が分割
- 各セッションは特定のフレーム範囲を持つ
- フレームごとのBBox数にばらつきあり（検出精度のシフト）

**後処理への示唆:**
- **同一フレーム内制約**: 各フレームで各選手は最大1回しか出現しない
- **選手数制約**: 試合全体で最大10名（交代を含む）
- **連続性**: 連続フレーム間では選手の多くが一致する（トラッキング情報の活用余地あり）

## 6. 画角（angle）分布

In [ ]:
# 訓練データとテストデータの画角分布
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 訓練データ
angle_counts_train = df_train['angle'].value_counts()
axes[0].bar(angle_counts_train.index, angle_counts_train.values, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Angle', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Training Data: Angle Distribution', fontsize=14)
axes[0].grid(axis='y', alpha=0.3)

# テストデータ
angle_counts_test = df_test['angle'].value_counts()
axes[1].bar(angle_counts_test.index, angle_counts_test.values, color='orange', edgecolor='black')
axes[1].set_xlabel('Angle', fontsize=12)
axes[1].set_ylabel('Sample Count', fontsize=12)
axes[1].set_title('Test Data: Angle Distribution', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("訓練データ:")
print(angle_counts_train)
print("\nテストデータ:")
print(angle_counts_test)
print("\n⚠️ テストデータはside画角のみ（訓練データはside/topの両方）")

## 7. BBox サイズ分布

In [ ]:
# BBoxのサイズ（幅・高さ）分布
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 訓練データ
axes[0, 0].hist(df_train['w'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Width (pixels)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Training Data: BBox Width Distribution', fontsize=14)
axes[0, 0].grid(axis='y', alpha=0.3)

axes[0, 1].hist(df_train['h'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Height (pixels)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Training Data: BBox Height Distribution', fontsize=14)
axes[0, 1].grid(axis='y', alpha=0.3)

# テストデータ
axes[1, 0].hist(df_test['w'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Width (pixels)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Test Data: BBox Width Distribution', fontsize=14)
axes[1, 0].grid(axis='y', alpha=0.3)

axes[1, 1].hist(df_test['h'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Height (pixels)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Test Data: BBox Height Distribution', fontsize=14)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("訓練データ BBox統計:")
print(df_train[['w', 'h']].describe())
print("\nテストデータ BBox統計:")
print(df_test[['w', 'h']].describe())

## 8. 画像サンプル確認

In [ ]:
# 訓練データ: 事前クロップ済み画像を確認
train_crop_dir = Path(DIR_TRAIN_CROPS)

print(f"訓練データクロップ画像ディレクトリ: {train_crop_dir}")
print(f"ディレクトリ存在: {train_crop_dir.exists()}")

if train_crop_dir.exists():
    crop_files = list(train_crop_dir.glob('*.jpg'))
    print(f"クロップ画像数: {len(crop_files)}")
    
    # ランダムに選手ごとに1枚ずつ表示
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    axes = axes.flatten()
    
    for label_id in range(11):  # 0-10の選手
        # 該当選手のサンプルを1つ取得
        sample = df_train[df_train['label_id'] == label_id].iloc[0]
        img_path = train_crop_dir / f"{sample.name}.jpg"
        
        if img_path.exists():
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[label_id].imshow(img)
            axes[label_id].set_title(f'Player {label_id}', fontsize=10)
            axes[label_id].axis('off')
        else:
            axes[label_id].text(0.5, 0.5, 'Not Found', ha='center', va='center')
            axes[label_id].axis('off')
    
    axes[-1].axis('off')  # 12番目は空白
    
    plt.suptitle('Training Data: Sample Images per Player', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 訓練データのクロップ画像が見つかりません")
    print("   preprocess_train_crops.ipynb を実行してください")

In [ ]:
# テストデータ: rel_path経由で画像を確認
test_crop_dir = Path(DIR_INPUT)

print(f"テストデータクロップ画像ベースディレクトリ: {test_crop_dir}")

# ランダムに12枚表示
fig, axes = plt.subplots(2, 6, figsize=(18, 6))
axes = axes.flatten()

sample_indices = np.random.choice(len(df_test), min(12, len(df_test)), replace=False)

for i, idx in enumerate(sample_indices):
    sample = df_test.iloc[idx]
    img_path = test_crop_dir / sample['rel_path']
    
    if img_path.exists():
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img)
        axes[i].set_title(f'Test {idx}\n{sample["angle"]}', fontsize=9)
        axes[i].axis('off')
    else:
        axes[i].text(0.5, 0.5, 'Not Found', ha='center', va='center')
        axes[i].axis('off')

plt.suptitle('Test Data: Random Sample Images', fontsize=16)
plt.tight_layout()
plt.show()

## 9. CV戦略の検討

In [ ]:
# Group K-Fold用のグループ列を作成
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"グループ数: {df_train['group'].nunique()}")
print(f"平均サンプル数/グループ: {len(df_train) / df_train['group'].nunique():.1f}")
print(f"\nグループごとのサンプル数（上位10件）:")
print(df_train['group'].value_counts().reset_index(drop=False).sort_values(by='group'))

In [ ]:
# グループごとの選手分布を確認（リーク防止のため）
print("各グループに含まれる選手数:")
group_label_counts = df_train.groupby('group')['label_id'].nunique().sort_values(ascending=False)
print(group_label_counts.head(10))

# グループサイズの分布
plt.figure(figsize=(14, 5))
group_sizes = df_train['group'].value_counts().sort_values(ascending=False)
plt.bar(range(len(group_sizes)), group_sizes.values, color='steelblue', alpha=0.7)
plt.xlabel('Group Index (sorted by size)', fontsize=12)
plt.ylabel('Sample Count', fontsize=12)
plt.title('Group Size Distribution (for Group K-Fold)', fontsize=14)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 10. まとめ

### データ特徴
1. **訓練データ**: 全選手のサンプルあり、side/top両方の画角
2. **テストデータ**: side画角のみ、複数session
3. **BBoxサイズ**: 訓練/テストで分布が異なる可能性

### CV戦略
- **Group K-Fold**: `quarter_session`でグループ化
- **理由**: 同じシーンの画像を訓練/検証に分けないため（リーク防止）

### 注意点
- テストデータは**side画角のみ**なので、topで過学習しないよう注意
- sessionが0のみの訓練データと、複数sessionのテストデータで分布が異なる
- unknown選手（-1）の判定が重要

## テスト環境を模倣したデータ拡張

学習データとテストデータのギャップを埋めるための拡張を可視化：
1. BBox shift: テストデータのBBox不正確を模倣
2. Partial crop: IoU < 0.5（一部のみ写っている）を模倣
3. Lighting shift: 時系列による照明変化を模倣

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random

# テスト環境を模倣した拡張関数
def bbox_shift_augmentation(image, max_shift=0.15):
    """BBoxの位置ずれを模倣（テストデータのBBox不正確を再現）"""
    h, w = image.shape[:2]
    
    # ランダムにシフト量を決定
    shift_x = int(w * random.uniform(-max_shift, max_shift))
    shift_y = int(h * random.uniform(-max_shift, max_shift))
    
    # シフト後のBBox計算（はみ出さないように調整）
    new_x1 = max(0, shift_x)
    new_y1 = max(0, shift_y)
    new_x2 = min(w, w + shift_x)
    new_y2 = min(h, h + shift_y)
    
    # 元画像からクロップ
    crop_x1 = max(0, -shift_x)
    crop_y1 = max(0, -shift_y)
    crop_x2 = crop_x1 + (new_x2 - new_x1)
    crop_y2 = crop_y1 + (new_y2 - new_y1)
    
    cropped = image[crop_y1:crop_y2, crop_x1:crop_x2]
    
    # リサイズして元のサイズに戻す
    resized = cv2.resize(cropped, (w, h))
    return resized

def partial_crop_augmentation(image, keep_ratio=0.6):
    """部分的な切り取り（IoU < 0.5を模倣）"""
    h, w = image.shape[:2]
    
    # ランダムに切り取る辺を選択
    side = random.choice(['top', 'bottom', 'left', 'right'])
    
    crop_amount = 1 - keep_ratio
    
    if side == 'top':
        crop_h = int(h * crop_amount)
        cropped = image[crop_h:, :]
    elif side == 'bottom':
        crop_h = int(h * crop_amount)
        cropped = image[:-crop_h, :]
    elif side == 'left':
        crop_w = int(w * crop_amount)
        cropped = image[:, crop_w:]
    else:  # right
        crop_w = int(w * crop_amount)
        cropped = image[:, :-crop_w]
    
    # リサイズして元のサイズに戻す
    resized = cv2.resize(cropped, (w, h))
    return resized

def lighting_shift_augmentation(image, intensity=0.3):
    """照明変化（時系列ギャップを模倣）"""
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(np.float32)
    
    # V（明度）チャンネルをランダムに変更
    brightness_factor = 1 + random.uniform(-intensity, intensity)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * brightness_factor, 0, 255)
    
    # 色温度の変化（H チャンネル）
    hue_shift = random.uniform(-10, 10)
    hsv[:, :, 0] = (hsv[:, :, 0] + hue_shift) % 180
    
    rgb = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)
    return rgb

print("拡張関数定義完了")

In [ ]:
# サンプル画像を選択して拡張を可視化
sample_idx = 100  # 任意のサンプル

sample_row = df_train.iloc[sample_idx]
img_path = train_crop_dir / f"{sample_row.name}.jpg"
bbox_img = cv2.imread(img_path)


# 各種拡張を適用
bbox_shift = bbox_shift_augmentation(bbox_img.copy(), max_shift=0.15)
partial_crop = partial_crop_augmentation(bbox_img.copy(), keep_ratio=0.6)
lighting_shift = lighting_shift_augmentation(bbox_img.copy(), intensity=0.3)

# 複数の拡張を組み合わせ
combined = bbox_shift_augmentation(bbox_img.copy(), max_shift=0.1)
combined = lighting_shift_augmentation(combined, intensity=0.2)

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle(f'テスト環境を模倣したデータ拡張の例\n選手ID: {sample_row["label_id"]}, Quarter: {sample_row["quarter"]}, Angle: {sample_row["angle"]}', 
             fontsize=14, fontweight='bold')

# オリジナル
axes[0, 0].imshow(bbox_img)
axes[0, 0].set_title('オリジナル（学習データ）\n完璧なBBox', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# BBox shift
axes[0, 1].imshow(bbox_shift)
axes[0, 1].set_title('BBox位置ずれ\nmax_shift=15%', fontsize=12)
axes[0, 1].axis('off')

# Partial crop
axes[0, 2].imshow(partial_crop)
axes[0, 2].set_title('部分的切り取り\nkeep_ratio=60% (IoU<0.5模倣)', fontsize=12)
axes[0, 2].axis('off')

# Lighting shift
axes[1, 0].imshow(lighting_shift)
axes[1, 0].set_title('照明変化\nintensity=30%', fontsize=12)
axes[1, 0].axis('off')

# Combined
axes[1, 1].imshow(combined)
axes[1, 1].set_title('組み合わせ\n(BBox shift + Lighting)', fontsize=12)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"\n画像情報:")
print(f"  オリジナルサイズ: {bbox_img.shape[:2]}")
print(f"  選手ID: {sample_row['label_id']}")
print(f"  Quarter: {sample_row['quarter']}")
print(f"  Angle: {sample_row['angle']}")

In [ ]:
# 複数サンプルでの拡張効果を確認
fig, axes = plt.subplots(5, 5, figsize=(15, 15))
fig.suptitle('ランダムサンプルでのデータ拡張効果（各行: 異なる選手）', fontsize=14, fontweight='bold')

# 5人の異なる選手からランダムサンプル
np.random.seed(42)
sample_players = np.random.choice(df_train['label_id'].unique(), 5, replace=False)

for i, player_id in enumerate(sample_players):
    player_data = df_train[df_train['label_id'] == player_id]
    sample_row = player_data.sample(1).iloc[0]
    
    # 画像読み込み
    image_path = train_crop_dir / f"{sample_row.name}.jpg"
    bbox_img = cv2.imread(image_path)
    bbox_img = cv2.cvtColor(bbox_img, cv2.COLOR_BGR2RGB)
    
    # オリジナル
    axes[i, 0].imshow(bbox_img)
    axes[i, 0].set_title(f'選手{player_id}\nオリジナル', fontsize=10)
    axes[i, 0].axis('off')
    
    # BBox shift (弱)
    aug1 = bbox_shift_augmentation(bbox_img.copy(), max_shift=0.1)
    axes[i, 1].imshow(aug1)
    axes[i, 1].set_title('BBox shift\n(10%)', fontsize=10)
    axes[i, 1].axis('off')
    
    # BBox shift (強)
    aug2 = bbox_shift_augmentation(bbox_img.copy(), max_shift=0.2)
    axes[i, 2].imshow(aug2)
    axes[i, 2].set_title('BBox shift\n(20%)', fontsize=10)
    axes[i, 2].axis('off')
    
    # Partial crop
    aug3 = partial_crop_augmentation(bbox_img.copy(), keep_ratio=0.50)
    axes[i, 3].imshow(aug3)
    axes[i, 3].set_title('Partial crop\n(65%)', fontsize=10)
    axes[i, 3].axis('off')
    
    # Combined (実戦想定)
    aug4 = bbox_shift_augmentation(bbox_img.copy(), max_shift=0.12)
    aug4 = lighting_shift_augmentation(aug4, intensity=0.25)
    axes[i, 4].imshow(aug4)
    axes[i, 4].set_title('Combined\n(実戦想定)', fontsize=10)
    axes[i, 4].axis('off')

plt.tight_layout()
plt.show()

print("\n拡張パラメータの説明:")
print("  BBox shift: テストデータのBBox位置ずれを模倣")
print("  Partial crop: IoU < 0.5（一部だけ写っている）を模倣")
print("  Lighting shift: 時系列による照明・色温度変化を模倣")
print("  Combined: 実戦で想定される複数の変化を同時適用")

### 推奨パラメータと適用確率

検証データに対してテスト環境を模倣した拡張を適用する場合の推奨設定：

| 拡張 | パラメータ | 適用確率 | 目的 |
|------|-----------|---------|------|
| BBox shift | max_shift=0.15 | 30% | テストのBBox位置ずれを模倣 |
| Partial crop | keep_ratio=0.6 | 20% | IoU < 0.5（部分的に写る）を模倣 |
| Lighting shift | intensity=0.3 | 30% | 時系列による照明変化を模倣 |
| Combined | shift=0.12 + light=0.25 | 20% | 複数の変化を同時適用 |

**重要:** 学習データには通常の拡張のみ適用し、検証データにのみテスト模倣拡張を適用する